In [540]:
from pytube import YouTube
import pandas as pd
import numpy as np
import os
import moviepy.editor
import time
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip

In [541]:
directory = 'H:\Digital_Futures_Projects\CapstoneProject\PracticeData\MS-ASL'

In [542]:
filename = directory+'\MSASL_train.json'

In [543]:
train_vids = pd.read_json(filename)

In [544]:
train_vids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16054 entries, 0 to 16053
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   org_text    16054 non-null  object 
 1   clean_text  16054 non-null  object 
 2   start_time  16054 non-null  float64
 3   signer_id   16054 non-null  int64  
 4   signer      16054 non-null  int64  
 5   start       16054 non-null  int64  
 6   end         16054 non-null  int64  
 7   file        16054 non-null  object 
 8   label       16054 non-null  int64  
 9   height      16054 non-null  int64  
 10  fps         16054 non-null  float64
 11  end_time    16054 non-null  float64
 12  url         16054 non-null  object 
 13  text        16054 non-null  object 
 14  box         16054 non-null  object 
 15  width       16054 non-null  int64  
 16  review      5169 non-null   float64
dtypes: float64(4), int64(7), object(6)
memory usage: 2.1+ MB


In [545]:
add_words = ['camera','soap','cookie','cereal','bacon','orange','apple','banana','onion','bread','milk','butter'
        ,'cheese','brave','smart','popcorn','warm','cost','cold','lie','blue','television','pink','color'
        ,'snob','cry','bed','sheep','medicine','disgusted','bedroom','kitchen','cup','several','clock','juice'
            ,'girl','purple','monday','tuesday','wednesday','thursday','friday','saturday','sunday','hat','tiger','old','fishing','painting','sick'
             ,'motorcycle','spider','lion','deer','monkey','cat','mouse','fox','whale','turtle','horse','fish','dog','frog','pig','elephant','duck'
            ,'rain','sun','rain','winter','summer','fire','home','cousin','science','breakfast','afternoon','boring','easter','shock','nervous','dark','light','autumn','spring'
             ,'city','hippo','chicken','sore throat','umbrella']

In [546]:
len(add_words)

92

In [547]:
word_list = pd.read_csv('sign_words.csv')

In [548]:
word_list.head()

,rank,word
0,1,again
1,2,also
2,3,ask
3,4,asl - american sign language
4,5,because


In [549]:
word_list[word_list['word']=='like (feeling)'] = 'enjoy'
word_list[word_list['word']=='asl - american sign language'] = 'asl'
word_list[word_list['word']=='close (near)'] = 'near'
word_list[word_list['word']=='close (shut)'] = 'close'
word_list[word_list['word']=='lose'] = 'forget'
word_list[word_list['word']=='email address'] = 'email'
word_list[word_list['word']=='find out'] = 'find'
word_list[word_list['word']=='toilet'] = 'bathroom'
word_list[word_list['word']=='pen'] = 'pencil'
word_list[word_list['word']=='begin'] = 'start'
word_list[word_list['word']=='little'] = 'small'
word_list[word_list['word']=='done'] = 'finish'
word_list[word_list['word']=='gift'] = 'surprise'
word_list[word_list['word']=='idea'] = 'plan'
word_list[word_list['word']=='look for'] = 'search'
word_list[word_list['word']=='grow'] = 'fat'
word_list[word_list['word']=='everything'] = 'all'

In [550]:
word_list = word_list['word'].to_list()

In [551]:
word_list = [str(x) for x in word_list]
type(word_list[0])

str

In [552]:
word_list = word_list+add_words

In [553]:
word_list.index('thank you')

74

In [554]:
train_vids[train_vids['clean_text']=='thank you'].head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
511,Thank you,thank you,149.516,30,-1,4481,4571,Barista ASL Vocabulary,75,360,29.97,152.519,https://www.youtube.com/watch?v=Cgh1DXAQBuI,thanks,"[0.07821321487426701, 0.316317319869995, 0.998...",640,NaN
729,Thank you,thank you,497.330,43,21,14905,15075,Unit 1 Vocabulary(2),75,360,29.97,503.002,https://www.youtube.com/watch?v=7YYB3BEoksc,thanks,"[0.042467057704925, 0.0047622621059410005, 1.0...",640,NaN
1200,thank you,thank you,0.000,2,58,0,89,Thank you,75,360,17.13,5.196,https://www.youtube.com/watch?v=QUF1JHzBXhw,thanks,"[0.0, 0.23549106717109602, 1.0, 0.879804968833...",640,NaN
1956,THANK YOU,thank you,446.680,98,20,13387,13566,CCHS ASL Unit 1,75,720,29.97,452.652,https://www.youtube.com/watch?v=eXpXg4q-qEQ&t=...,thanks,"[0.108920216560363, 0.31048071384429904, 0.900...",960,NaN
3011,THAN K-YOU,thank you,61.328,24,1,1838,1901,Permission,75,720,29.97,63.430,https://www.youtube.com/watch?v=SI_7UivPW_I,thanks,"[0.08984565734863201, 0.42944115400314303, 0.9...",1280,NaN


In [555]:
train_vids.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,match [light-a-MATCH],match,0.0,0,0,0,83,match light-a-MATCH,830,360,30.00,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.057544618844985004, 0.21637457609176602, 1....",640,NaN
1,FAIL,fail,0.0,0,-1,0,74,FAIL,542,360,25.00,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.06577941775321901, 0.167171776294708, 0.939...",480,NaN
2,laugh,laugh,0.0,4,26,0,31,SignSchool Laugh with Legs 2,312,360,29.97,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.131885945796966, 0.32334136962890603, 1.0, ...",640,NaN
3,BOOK,book,0.0,0,-1,0,66,BOOK(3),38,360,25.00,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.055698871612548, 0.25173279643058705, 0.996...",480,NaN
4,sign-language,sign language,0.0,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360,29.97,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.039043992757797005, 0.24198183417320202, 1....",640,NaN


In [556]:
train_vids['start_time'] = train_vids['start']/train_vids['fps']

In [557]:
train_vids['file_name'] = train_vids['clean_text']+train_vids.index.astype(str)

In [558]:
train_vids[train_vids['start']!=0]

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,file_name
5,school,school,1.101101,1,44,33,110,ASL Vocabulary school,10,360,29.970,3.670,https://www.youtube.com/watch?v=1AyT77LqJzQ,school,"[0.0, 0.07651948928833001, 1.0, 1.0]",640,1.0,school5
6,school,school,4.671338,1,44,140,206,ASL Vocabulary school,10,360,29.970,6.874,https://www.youtube.com/watch?v=1AyT77LqJzQ,school,"[0.0, 0.07651948928833001, 1.0, 1.0]",640,1.0,school6
11,telephone,phone,2.461974,2,81,73,123,Telephone,120,360,29.651,4.148,https://www.youtube.com/watch?v=HPz_C5XM4o4,phone,"[0.0, 0.133232593536376, 1.0, 0.8751190900802611]",640,1.0,phone11
12,telephone,phone,4.991400,2,81,148,196,Telephone,120,360,29.651,6.610,https://www.youtube.com/watch?v=HPz_C5XM4o4,phone,"[0.0, 0.133232593536376, 1.0, 0.8751190900802611]",640,1.0,phone12
17,rainbow,rainbow,1.803969,144,-1,54,165,rainbow - ASL sign for rainbow,669,360,29.934,5.512,https://www.youtube.com/watch?v=WeAFuzYTdtU,rainbow,"[0.004803866147994, 0.0, 1.0, 0.9657534956932061]",480,1.0,rainbow17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16045,CLOSE,close,518.518519,32,17,15540,15689,ASL 1 Unit 1 Vocabulary,549,360,29.970,523.490,https://www.youtube.com/watch?v=fNg_sJ9f8EI,close,"[0.025470584630966003, 0.27127972245216303, 1....",640,NaN,close16045
16046,CHAIR,sit,523.857191,32,17,15700,15849,ASL 1 Unit 1 Vocabulary,18,360,29.970,528.828,https://www.youtube.com/watch?v=fNg_sJ9f8EI,sit,"[0.026675939559936003, 0.33721128106117204, 1....",640,NaN,sit16046
16047,READ,read,529.262596,32,17,15862,16011,ASL 1 Unit 1 Vocabulary,82,360,29.970,534.234,https://www.youtube.com/watch?v=fNg_sJ9f8EI,read,"[0.011600971221923, 0.33937367796897805, 1.0, ...",640,NaN,read16047
16048,THROW,throw,534.534535,32,17,16020,16168,ASL 1 Unit 1 Vocabulary,829,360,29.970,539.472,https://www.youtube.com/watch?v=fNg_sJ9f8EI,throw,"[0.024654358625411002, 0.12953698635101302, 1....",640,NaN,throw16048


In [559]:
clean_text_df = train_vids[(train_vids['clean_text'].isin(word_list))]
clean_text_df[clean_text_df['clean_text']=='thank you']

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,file_name
511,Thank you,thank you,149.516183,30,-1,4481,4571,Barista ASL Vocabulary,75,360,29.970,152.519,https://www.youtube.com/watch?v=Cgh1DXAQBuI,thanks,"[0.07821321487426701, 0.316317319869995, 0.998...",640,NaN,thank you511
729,Thank you,thank you,497.330664,43,21,14905,15075,Unit 1 Vocabulary(2),75,360,29.970,503.002,https://www.youtube.com/watch?v=7YYB3BEoksc,thanks,"[0.042467057704925, 0.0047622621059410005, 1.0...",640,NaN,thank you729
1200,thank you,thank you,0.000000,2,58,0,89,Thank you,75,360,17.130,5.196,https://www.youtube.com/watch?v=QUF1JHzBXhw,thanks,"[0.0, 0.23549106717109602, 1.0, 0.879804968833...",640,NaN,thank you1200
1956,THANK YOU,thank you,446.680013,98,20,13387,13566,CCHS ASL Unit 1,75,720,29.970,452.652,https://www.youtube.com/watch?v=eXpXg4q-qEQ&t=...,thanks,"[0.108920216560363, 0.31048071384429904, 0.900...",960,NaN,thank you1956
3011,THAN K-YOU,thank you,61.327995,24,1,1838,1901,Permission,75,720,29.970,63.430,https://www.youtube.com/watch?v=SI_7UivPW_I,thanks,"[0.08984565734863201, 0.42944115400314303, 0.9...",1280,NaN,thank you3011
3012,THAN K-YOU,thank you,63.496830,24,1,1903,1999,Permission,75,720,29.970,66.700,https://www.youtube.com/watch?v=SI_7UivPW_I,thanks,"[0.087675869464874, 0.43241107463836603, 0.994...",1280,NaN,thank you3012
3013,THAN K-YOU,thank you,66.766767,24,1,2001,2032,Permission,75,720,29.970,67.801,https://www.youtube.com/watch?v=SI_7UivPW_I,thanks,"[0.08057522773742601, 0.421390324831008, 0.999...",1280,NaN,thank you3013
3169,thank you,thank you,0.000000,229,45,0,77,thank you in American Sign Language,75,270,29.917,2.574,www.youtube.com/watch?v=7XHOmZYiBew,thanks,"[0.009470313787460001, 0.24607436358928603, 1....",480,NaN,thank you3169
3952,THANK YOU,thank you,19.120000,94,2,478,572,First 100 Social Function,75,720,25.000,22.880,https://www.youtube.com/watch?v=6XcxbPfP5YQ,thanks,"[0.09059950709342901, 0.14841678738594002, 1.0...",1280,NaN,thank you3952
4031,thank you,thank you,0.000000,110,49,0,91,thank you 2,75,360,29.970,3.036,www.youtube.com/watch?v=sH87G_Bz_nA,thanks,"[0.087902337312698, 0.293206930160522, 1.0, 0....",640,NaN,thank you4031


In [560]:
### checking for synonyms
train_vids[train_vids['clean_text']=='science'].head(1)

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,file_name
595,science,science,0.0,2,58,0,50,Science,287,360,29.644,1.687,https://www.youtube.com/watch?v=rBZlr8EEVHs,science,"[0.0, 0.0, 1.0, 0.981499671936035]",640,1.0,science595


In [561]:
clean_text_df[clean_text_df['clean_text']=='science'].head(1)

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,file_name
595,science,science,0.0,2,58,0,50,Science,287,360,29.644,1.687,https://www.youtube.com/watch?v=rBZlr8EEVHs,science,"[0.0, 0.0, 1.0, 0.981499671936035]",640,1.0,science595


In [562]:
word_list.index('find')

233

In [563]:
list(set(word_list).symmetric_difference(set(clean_text_df['clean_text'].to_list())))

['struggle',
 'against',
 'yourself',
 'nearby',
 'anything',
 'say',
 'share',
 'prefer',
 'attitude',
 'worse',
 'she',
 'signing',
 'bring',
 'texting',
 'hold',
 'allow',
 'airport',
 'receive',
 'signing fluently',
 'until',
 'agree',
 'other',
 'include',
 'thing',
 'guess',
 'announce',
 'story',
 'best',
 'by oneself',
 'sure',
 'doubt',
 'basic',
 'average',
 'avoid',
 'approve',
 'almost',
 'because',
 'wash',
 'word',
 'food',
 'than',
 'service',
 'imply',
 'away',
 'overlook',
 'life',
 'clean up',
 'try',
 'keep',
 'problem',
 'something',
 'enough']

In [564]:
len(clean_text_df['clean_text'].value_counts())

300

In [565]:
clean_text_df

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,file_name
3,BOOK,book,0.000000,0,-1,0,66,BOOK(3),38,360,25.000,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.055698871612548, 0.25173279643058705, 0.996...",480,NaN,book3
5,school,school,1.101101,1,44,33,110,ASL Vocabulary school,10,360,29.970,3.670,https://www.youtube.com/watch?v=1AyT77LqJzQ,school,"[0.0, 0.07651948928833001, 1.0, 1.0]",640,1.0,school5
6,school,school,4.671338,1,44,140,206,ASL Vocabulary school,10,360,29.970,6.874,https://www.youtube.com/watch?v=1AyT77LqJzQ,school,"[0.0, 0.07651948928833001, 1.0, 1.0]",640,1.0,school6
7,easter,easter,0.000000,2,58,0,116,Easter,794,360,29.595,3.920,https://www.youtube.com/watch?v=SVWABYmFdhs,easter,"[0.0, 0.12271708250045701, 1.0, 0.85962688922882]",640,NaN,easter7
8,Boring,boring,0.000000,13,-1,0,71,ASL Boring,46,360,25.000,2.840,https://www.youtube.com/watch?v=CYx7qm62Zwo,bored,"[0.05692723393440201, 0.26684892177581704, 0.9...",640,NaN,boring8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16046,CHAIR,sit,523.857191,32,17,15700,15849,ASL 1 Unit 1 Vocabulary,18,360,29.970,528.828,https://www.youtube.com/watch?v=fNg_sJ9f8EI,sit,"[0.026675939559936003, 0.33721128106117204, 1....",640,NaN,sit16046
16047,READ,read,529.262596,32,17,15862,16011,ASL 1 Unit 1 Vocabulary,82,360,29.970,534.234,https://www.youtube.com/watch?v=fNg_sJ9f8EI,read,"[0.011600971221923, 0.33937367796897805, 1.0, ...",640,NaN,read16047
16049,WRITE,write,539.939940,32,17,16182,16301,ASL 1 Unit 1 Vocabulary,53,360,29.970,543.910,https://www.youtube.com/watch?v=fNg_sJ9f8EI,write,"[0.032177031040191005, 0.31613153219223, 1.0, ...",640,NaN,write16049
16050,hot,hot,0.000000,2,42,0,127,Hot,135,360,29.657,4.282,https://www.youtube.com/watch?v=MkXUHhsMAns,hot,"[0.000367134809494, 0.20649650692939703, 1.0, ...",640,NaN,hot16050


In [566]:
train_vids = clean_text_df

In [579]:
train_vids = train_vids.reset_index()

In [568]:
len(train_vids)

6254

In [576]:
def video_setup(vid_url,start_time,end_time,video_name,error_list,clean_text):
    
    try:
        yt = YouTube(vid_url)
        yt = yt.streams.filter(progressive=True, file_extension='mp4').desc().order_by('resolution').first()
        yt.download(output_path='pre_processed_videos\\',filename=str(clean_text))
        VideoFileClip('pre_processed_videos\\'+str(clean_text)+'.mp4').subclip(start_time, end_time).write_videofile('training_videos\\'+str(clean_text)+'_clipped.mp4')
    except:
        print('Video No longer available')
        error_list.append(str(clean_text))

In [577]:
unav_vids = []

In [ ]:
for i in range(len(train_vids)):
    vid_url = train_vids['url'][i]
    start_time = train_vids['start_time'][i]
    end_time = train_vids['end_time'][i]
    video_name = train_vids['org_text'][i]
    clean_text = train_vids['file_name'][i]
    video_setup(vid_url,start_time,end_time,video_name,unav_vids,clean_text)
print(unav_vids)

chunk:   0%|                                                                          | 0/59 [00:00<?, ?it/s, now=None]

Moviepy - Building video training_videos\book3_clipped.mp4.
MoviePy - Writing audio in book3_clippedTEMP_MPY_wvf_snd.mp3


t:   0%|                                                                              | 0/66 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video training_videos\book3_clipped.mp4



Moviepy - Done !
Moviepy - video ready training_videos\book3_clipped.mp4
Video No longer available
Video No longer available


...

t:   0%|                                                                              | 0/75 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video training_videos\shock27_clipped.mp4



Moviepy - Done !
Moviepy - video ready training_videos\shock27_clipped.mp4


chunk:   0%|                                                                          | 0/65 [00:00<?, ?it/s, now=None]

Moviepy - Building video training_videos\breakfast30_clipped.mp4.
MoviePy - Writing audio in breakfast30_clippedTEMP_MPY_wvf_snd.mp3
